In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from PIL import Image
from tqdm import tqdm

root_dir = "../"
data_dir = os.path.join(root_dir, "data")
annotation_dir = os.path.join(data_dir, "annotations")
explanation_dir = os.path.join(root_dir, "explanations")
figure_dir = os.path.join(data_dir, "figures", "explanations")
gt_figure_dir = os.path.join(figure_dir, "ground_truth")
os.makedirs(gt_figure_dir, exist_ok=True)

gt_df = []
for sequence_name in ["daySequence1", "daySequence2"]:
    _df = pd.read_csv(
        os.path.join(annotation_dir, sequence_name, "frameAnnotationsBOX.csv"), sep=";"
    )
    gt_df.append(_df)
gt_df = pd.concat(gt_df, axis=0)
gt_df["ImageName"] = gt_df.apply(
    lambda row: os.path.basename(row["Filename"]).split(".")[0], axis=1
)
gt_df["Label"] = gt_df.apply(
    lambda row: 1 if "go" in row["Annotation tag"] else 0, axis=1
)
gt_df.set_index("ImageName", inplace=True)

exp_mapper = {
    "gradcam": r"Grad-CAM",
    "lime": r"LIME",
    "gradexp": r"GradientExp",
    "deepexp": r"DeepExp",
    "partexp/500": r"PartitionExp ($m = 500$)",
    "partexp/128": r"PartitionExp ($m = 128$)",
    "partexp/64": r"PartitionExp ($m = 64$)",
    "partexp/32": r"PartitionExp ($m = 32$)",
    "partexp/16": r"PartitionExp ($m = 16$)",
    "hexp/absolute_0": r"h-Shap ($\tau = 0$)",
    "hexp/relative_70": r"h-Shap ($\tau=70\%$)",
}

true_positive = np.load(os.path.join(explanation_dir, "true_positive_explained.npy"))

In [ ]:
sns.set_context("paper", font_scale=1.5)

for i, image_path in enumerate(tqdm(true_positive[:5])):
    image = Image.open(os.path.join(root_dir, image_path))

    _, ax = plt.subplots(1, 1)
    ax.imshow(image)
    ax.axes.xaxis.set_visible(False)
    ax.axes.yaxis.set_visible(False)
    ax.set_title("image")

    image_name = os.path.basename(image_path).split(".")[0]
    annotations = gt_df.loc[[image_name]]
    for _, annotation in annotations.iterrows():
        if annotation["Label"] == 1:
            upper_left_c = int(annotation["Upper left corner X"])
            upper_left_r = int(annotation["Upper left corner Y"])
            lower_right_c = int(annotation["Lower right corner X"])
            lower_right_r = int(annotation["Lower right corner Y"])
            w = abs(lower_right_c - upper_left_c)
            h = abs(lower_right_r - upper_left_r)
            bbox = patches.Rectangle(
                (upper_left_c, upper_left_r),
                w,
                h,
                linewidth=2,
                edgecolor="g",
                facecolor="none",
            )
            ax.add_patch(bbox)

    plt.savefig(os.path.join(gt_figure_dir, f"{image_name}.jpg"), bbox_inches="tight")
    plt.savefig(os.path.join(gt_figure_dir, f"{image_name}.pdf"), bbox_inches="tight")
    plt.close()

    for exp_name, exp_title in exp_mapper.items():
        explainer_dir = os.path.join(explanation_dir, exp_name)
        explainer_figure_dir = os.path.join(figure_dir, exp_name)
        os.makedirs(explainer_figure_dir, exist_ok=True)

        explanation = np.load(os.path.join(explainer_dir, f"{image_name}.npy"))
        _max = np.amax(np.abs(explanation.flatten()))

        _, ax = plt.subplots(1, 1)
        ax.imshow(explanation, cmap="bwr", vmin=-_max, vmax=_max)
        ax.axes.xaxis.set_visible(False)
        ax.axes.yaxis.set_visible(False)
        ax.set_title(exp_title)

        plt.savefig(
            os.path.join(explainer_figure_dir, f"{image_name}.jpg"), bbox_inches="tight"
        )
        plt.savefig(
            os.path.join(explainer_figure_dir, f"{image_name}.pdf"), bbox_inches="tight"
        )
        plt.close()